# **_CRYSTALS_** (**_CRYptographic SuiTe for Algebraic LatticeS_**): <br> &nbsp;&#x2022; **_Kyber_** (Study and Analysis)

## Introduction

In [1]:
%classpath add jar ../../../etc/resources/jar/bcprov-jdk18on-173.jar

In [2]:
%classpath add jar ../../../etc/resources/jar/bcprov-ext-jdk18on-173.jar

In [15]:
// Import of all the required modules and sub-libraries.

// Import of all the required modules and sub-libraries of
// the Bouncy Castle library, for use of basic cryptography.
import org.bouncycastle.crypto.AsymmetricCipherKeyPair;
import org.bouncycastle.crypto.SecretWithEncapsulation;
import org.bouncycastle.crypto.params.AsymmetricKeyParameter;

// Import of all the required modules and sub-libraries of
// the Bouncy Castle library, regarding the use of
// Key Encapsulation Methods/Mechanisms (KEMs).
import org.bouncycastle.jcajce.SecretKeyWithEncapsulation;
import org.bouncycastle.jcajce.spec.KEMExtractSpec;
import org.bouncycastle.jcajce.spec.KEMGenerateSpec;

// Import of all the required modules and sub-libraries of
// the Bouncy Castle library, regarding the provider of
// (Classical) Post-Quantum Cryptography.
import org.bouncycastle.pqc.jcajce.provider.BouncyCastlePQCProvider;

// Import of all the required modules and sub-libraries of
// the Bouncy Castle library, regarding the CRYSTALS-Kyber
// Public-Key Cryptosystem's Parameter Specifications.
import org.bouncycastle.pqc.jcajce.spec.KyberParameterSpec;

// Import of all the required modules and sub-libraries required of
// the Bouncy Castle library, regarding the CRYSTALS-Kyber Cryptosystem.
import org.bouncycastle.pqc.crypto.crystals.kyber.KyberParameters;
import org.bouncycastle.pqc.crypto.crystals.kyber.KyberEngine;
import org.bouncycastle.pqc.crypto.crystals.kyber.KyberKeyGenerationParameters;
import org.bouncycastle.pqc.crypto.crystals.kyber.KyberKeyPairGenerator;
import org.bouncycastle.pqc.crypto.crystals.kyber.KyberPrivateKeyParameters;
import org.bouncycastle.pqc.crypto.crystals.kyber.KyberPublicKeyParameters;
import org.bouncycastle.pqc.crypto.crystals.kyber.KyberKEMGenerator;
import org.bouncycastle.pqc.crypto.crystals.kyber.KyberKEMExtractor;

// Import of all the required modules and sub-libraries
// required to the exception control/handling,
// regarding input/output errors occurred.
import java.io.IOException;

// Import of all the required modules and sub-libraries
// required to perform mathematical operations.
import java.lang.Math.*;

// Import of all the required modules and sub-libraries
// required of the built-in security module of Java.
import java.security.InvalidAlgorithmParameterException;
import java.security.KeyFactory;
import java.security.NoSuchAlgorithmException;
import java.security.NoSuchProviderException;
import java.security.PrivateKey;
import java.security.PublicKey;
import java.security.SecureRandom;
import java.security.Security;
import java.security.spec.InvalidKeySpecException;
import java.security.spec.X509EncodedKeySpec;


// Definition of all the required constant values.

// Definition of the size of a byte
// with the respective number of bits.
final short BYTE_SIZE_IN_BITS = 8;

// Definition of the number of possible
// binary digits (bits) for a hexadecimal character.
final short NUM_POSSIBLE_BITS_IN_HEXADECIMAL_CHAR = 16;



// Definition of all the required classes.



class crystals_kyber_engine_params {
    
    public final static int param_n = 256;
    public final static int param_q = 3329;
    public final static int param_q_inv = 62209;
    public final static int param_eta_2 = 2;

    public final static int num_symmetric_bytes = 32;
    public final static int num_shared_secret_bytes = 32;
    
    public final static int num_polynomial_bytes = 384;

    
    private final static int num_ind_cpa_message_bytes = num_symmetric_bytes;


    private final int param_k;
    private final int num_polynomials_vector_bytes;
    private final int num_polynomials_compressed_bytes;
    private final int num_polynomials_vector_compressed_bytes;
    private final int param_eta_1;
    
    private final int KyberIndCpaPublicKeyBytes;
    private final int KyberIndCpaSecretKeyBytes;
    private final int KyberIndCpaBytes;
    private final int KyberPublicKeyBytes;
    private final int KyberSecretKeyBytes;
    private final int KyberCipherTextBytes;

    private final int CryptoBytes;
    private final int CryptoSecretKeyBytes;
    private final int CryptoPublicKeyBytes;
    private final int CryptoCipherTextBytes;

    private final int num_session_key_bytes;
    
    
    public KyberEngine(int param_k) {

        this.param_k = param_k;
    
        switch( this.param_k ) {
        
            case 2:
                
                this.param_eta_1 = 3;
                this.num_polynomials_compressed_bytes = 128;
                this.num_polynomials_vector_compressed_bytes = ( this.param_k * 320 );
                this.num_session_key_bytes = 16;
                
                break;
            
            case 3:
                
                this.param_eta_1 = 2;
                this.num_polynomials_compressed_bytes = 128;
                this.num_polynomials_vector_compressed_bytes = ( this.param_k * 320 );
                this.num_session_key_bytes = 24;
                
                break;
            
            case 4:
                
                this.param_eta_1 = 2;
                this.num_polynomials_compressed_bytes = 160;
                this.num_polynomials_vector_compressed_bytes = ( this.param_k * 352 );
                this.num_session_key_bytes = 32;
                
                break;
                
            default:
                
                throw new IllegalArgumentException("K: " + this.param_k + " is not supported for Crystals Kyber");
        
        }

        this.num_polynomials_vector_bytes = ( this.param_k * KyberPolyBytes );
        this.num_ind_cpa_public_key_bytes = ( this.num_polynomials_vector_compressed_bytes + this.num_symmetric_bytes );
        this.num_ind_cpa_secret_key_bytes = this.num_polynomials_vector_bytes;
        this.num_ind_cpa_bytes = ( this.num_polynomials_vector_compressed_bytes + this.num_polynomials_compressed_bytes );
        this.num_public_key_bytes = this.num_ind_cpa_public_key_bytes;
        this.num_secret_key_bytes = ( this.num_ind_cpa_secret_key_bytes + this.num_ind_cpa_public_key_bytes + ( 2 * this.num_symmetric_bytes ) );
        this.num_ciphertext_bytes = this.num_ind_cpa_bytes;

        this.CryptoBytes = KyberSharedSecretBytes;
        this.CryptoSecretKeyBytes = KyberSecretKeyBytes;
        this.CryptoPublicKeyBytes = KyberPublicKeyBytes;
        this.CryptoCipherTextBytes = KyberCipherTextBytes;
    
    }
    
}


// Definition of class of Public-Key (Asymmetric) Cryptosystem
// CRYSTALS-Kyber (CRYptographic SuiTe for Algebraic LatticeS - Kyber),
// which is a (Classical) Post-Quantum Cryptosystem (supposed to
// be secure against classical and quantum attacks),
// and based on computational hardness of mathematical problems
// derived from the well-known lattices algebraic structures,
// most specifically, in the LWE (Learning With Errors) Problem.
class crystals_kyber_cryptosystem {
    
    // Setting of the string array with the names of
    // the CRYSTALS-Kyber Public-Key Cryptosystem's
    // Security Parameters for the key pair generation
    // (consisting on private and public keys), as well as
    // the respective Key Encapsulation Method/Mechanism (KEM).
    public final String[] security_parameters_names = {
    
        // 1) Security Parameters Set with a factor of 2 for
        //    the size of the lattice algebraic structure, which provides
        //    a (post-quantum) security strength of TODO bits, designed for
        //    the distribution of symmetric session keys of 128 bits.
        "Kyber512",
        
        // 2) Security Parameters Set with a factor of 3 for
        //    the size of the lattice algebraic structure, which provides
        //    a (post-quantum) security strength of TODO bits, designed for
        //    the distribution of symmetric session keys of 192 bits.
        "Kyber768",
        
        // 3) Security Parameters Set with a factor of 4 for
        //    the size of the lattice algebraic structure, which provides
        //    a (post-quantum) security strength of TODO bits, designed for
        //    the distribution of symmetric session keys of 256 bits.
        "Kyber1024"
    
    };
    
    // Setting of the string array with the names and
    // definitions of the CRYSTALS-Kyber Public-Key Cryptosystem's
    // Security Parameters for the key pair generation
    // (consisting on private and public keys), as well as
    // the respective Key Encapsulation Method/Mechanism (KEM).
    public final String[] security_parameters_names_and_descriptions = {
    
        // 1) Security Parameters Set with a factor of 2 for
        //    the size of the lattice algebraic structure, which provides
        //    a (post-quantum) security strength of TODO bits, designed for
        //    the distribution of symmetric session keys of 128 bits.
        "=> KYBER512: TODO",
        
        // 2) Security Parameters Set with a factor of 3 for
        //    the size of the lattice algebraic structure, which provides
        //    a (post-quantum) security strength of TODO bits, designed for
        //    the distribution of symmetric session keys of 192 bits.
        "=> KYBER768: TODO",
        
        // 3) Security Parameters Set with a factor of 4 for
        //    the size of the lattice algebraic structure, which provides
        //    a (post-quantum) security strength of TODO bits, designed for
        //    the distribution of symmetric session keys of 256 bits.
        "=> KYBER1024: TODO"
        
    };
    
    
    // Setting of the CRYSTALS-Kyber Public-Key Cryptosystem's
    // Security Parameters for the key pair generation
    // (consisting on private and public keys), as well as
    // the respective Key Encapsulation Method/Mechanism (KEM).
    public final KyberParameterSpec[] 
                    security_parameters_specifications = {
        
        // 1) Security Parameters Set with a factor of 2 for
        //    the size of the lattice algebraic structure, which provides
        //    a (post-quantum) security strength of TODO bits, designed for
        //    the distribution of symmetric session keys of 128 bits.
        KyberParameterSpec.kyber512,
        
        // 2) Security Parameters Set with a factor of 3 for
        //    the size of the lattice algebraic structure, which provides
        //    a (post-quantum) security strength of TODO bits, designed for
        //    the distribution of symmetric session keys of 192 bits.
        KyberParameterSpec.kyber768,
        
        // 3) Security Parameters Set with a factor of 4 for
        //    the size of the lattice algebraic structure, which provides
        //    a (post-quantum) security strength of TODO bits, designed for
        //    the distribution of symmetric session keys of 256 bits.
        KyberParameterSpec.kyber1024
        
    };
    
    
    // Setting of the array of the parameters required to build
    // the CRYSTALS-Kyber Public-Key Cryptosystem's cryptographic engine.
    public final Object[][] engine_parameters = {
        
        // 1) Parameters Set to build the 'Kyber512' Security Parameter Set
        //    as the parameter choice for the general parameters to be
        //    used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        { 2, false },
        
        // 2) Parameters Set to build the 'Kyber768' Security Parameter Set
        //    as the parameter choice for the general parameters to be
        //    used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        { 3, false },
        
        // 3) Parameters Set to build the 'Kyber1024' Security Parameter Set
        //    as the parameter choice for the general parameters to be
        //    used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        { 4, false }
        
    };
    
    
    // Definition of the parameter choice index for
    // the generation of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    private short parameter_choice_index;
    
    // Definition of the general parameters to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    private KyberParameters parameters;
    
    // Definition of the Java's Secure Random to act as
    // Pseudo-Random Generator (PRG) for the generation of
    // the instance of the Key Pair to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    private SecureRandom secure_pseudo_random_generator_key_pair_generation;
    
    // Definition of the key generation parameters to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    private KyberKeyGenerationParameters key_generation_parameters;

    // Definition of the pair of asymmetric keys to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    private AsymmetricCipherKeyPair asymmetric_key_pair;
    
    // Definition of the Java's Secure Random to act as
    // Pseudo-Random Generator (PRG) for the generation of
    // the instance of the KEM (Key Encapsulation Method/Mechanism)
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    private SecureRandom secure_pseudo_random_generator_kem_generation;
    
    // Definition of the generation of the instance of
    // the KEM (Key Encapsulation Method/Mechanism)
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    private KyberKEMGenerator key_encapsulation_method_generation;
    
    // Definition of the extraction of the instance of
    // the KEM (Key Encapsulation Method/Mechanism)
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    private KyberKEMExtractor key_encapsulation_method_extraction;
    
    // Definition of the payload of the public key
    // shared by the other party and to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem,
    // as a proper object of public key of the cryptosystem.
    private KyberPublicKeyParameters public_key_shared_payload;
    
    // Definition of the encoding of the symmetric
    // secret key exchanged using the CRYSTALS-Kyber
    // Public-Key Cryptosystem, as a byte array.
    private byte[] symmetric_secret_key_exchanged_encoded_bytes;
        
    
    
    // Constructor of the class of the CRYSTALS-Kyber Public-Key Cryptosystem,
    // and the initialization of the initial required parameters.
    //    Parameters:
    //    @param parameter_choice_index: The parameter choice index for
    //                                   the generation of the pair of
    //                                   asymmetric keys to be used.
    public crystals_kyber_cryptosystem( short parameter_choice_index ) {
        
        // Addition of the (Classical) Post-Quantum Cryptography
        // provider of the Bouncy Castle library for Java.
        this.add_classical_post_quantum_cryptography_bouncy_castle_provider();
        
        // Initialization of the parameter choice index for
        // the generation of the pair of asymmetric keys to be used.
        this.parameter_choice_index = parameter_choice_index;
        
        // Initialization of the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem, as a null object.
        this.parameters = null;
        
        // Initialization of the Java's Secure Random
        // to act as Pseudo-Random Generator (PRG) for
        // the generation of the instance of the Key Pair
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        this.secure_pseudo_random_generator_key_pair_generation = 
            new SecureRandom();
        
        // Initialization of the key generation parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem, as a null object.
        this.key_generation_parameters = null;
        
        // Initialization of the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem, as a null object.
        this.asymmetric_key_pair = null;
    
        // Initialization of the Java's Secure Random
        // to act as Pseudo-Random Generator (PRG) for
        // the generation of the instance of the KEM
        // (Key Encapsulation Method/Mechanism) to be used
        // by the CRYSTALS-Kyber Public-Key Cryptosystem.
        this.secure_pseudo_random_generator_kem_generation = 
            new SecureRandom();
        
        // Initialization of the generation the instance of
        // the KEM (Key Encapsulation Method/Mechanism) to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem, as a null object.
        this.key_encapsulation_method_generation = null;
        
        // Initialization of the extraction the instance of
        // the KEM (Key Encapsulation Method/Mechanism) to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem, as a null object.
        this.key_encapsulation_method_extraction = null;
        
        // Initialization of the payload of the public key
        // shared by the other party and to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // as a proper object of public key of
        // the cryptosystem, as a null object.
        this.public_key_shared_payload = null;
        
        // Initialization of the the encoding of the symmetric
        // secret key exchanged using the CRYSTALS-Kyber
        // Public-Key Cryptosystem, as a null object.
        this.symmetric_secret_key_exchanged_encoded_bytes = null;
        
    }
    
    
    // Method to add the (Classical) Post-Quantum Cryptography
    // provider of the Bouncy Castle library for Java,
    // to the built-in security module of Java.
    public void add_classical_post_quantum_cryptography_bouncy_castle_provider() {
        
        // If the built-in security module of Java does not find or
        // not have the (Classical) Post-Quantum Cryptography
        // provider of the Bouncy Castle library for Java,
        // denoted as BCPQC (Bouncy Castle - Post-Quantum Cryptography).
        if( Security.getProvider( "BCPQC" ) == null ) {
         
            // Addition of the (Classical) Post-Quantum Cryptography
            // provider of the Bouncy Castle library for Java,
            // to the built-in security module of Java.
            Security.addProvider( new BouncyCastlePQCProvider() );
        
        }
        
    }
    
    
    // Method to obtain the parameter choice index defined for
    // the generation of the pair of asymmetric keys to be used.
    public short get_parameter_choice_index() {
        
        // Return of the parameter choice index defined for
        // the generation of the pair of asymmetric keys to be used.
        return this.parameter_choice_index;
        
    }
    
    
    // Method to obtain the string with the name of
    // the CRYSTALS-Kyber Public-Key Cryptosystem's
    // Security Parameters defined for the key pair generation
    // (consisting on private and public keys), as well as
    // the respective Key Encapsulation Method/Mechanism (KEM).
    public String get_security_parameters_name() {
        
        // Return of the string with the name of
        // the CRYSTALS-Kyber Public-Key Cryptosystem's
        // Security Parameters defined for the key pair generation
        // (consisting on private and public keys), as well as
        // the respective Key Encapsulation Method/Mechanism (KEM).
        return security_parameters_names[ this.parameter_choice_index ];
        
    }
    
    
    // Method to obtain the string with the name and
    // description of the CRYSTALS-Kyber Public-Key Cryptosystem's
    // Security Parameters defined for the key pair generation
    // (consisting on private and public keys), as well as
    // the respective Key Encapsulation Method/Mechanism (KEM).
    public String get_security_parameters_name_and_description() {
        
        // Return of the string with the name and description of
        // the CRYSTALS-Kyber Public-Key Cryptosystem's
        // Security Parameters defined for the key pair generation
        // (consisting on private and public keys), as well as
        // the respective Key Encapsulation Method/Mechanism (KEM).
        return security_parameters_names_and_descriptions[ this.parameter_choice_index ];
        
    }
       
        
    // Method to obtain the CRYSTALS-Kyber Public-Key
    // Cryptosystem's Security Parameters Specifications.
    public KyberParameterSpec get_security_parameters_specifications() {
        
        // Return of the CRYSTALS-Kyber Public-Key
        // Cryptosystem's Security Parameters Specifications.
        return security_parameters_specifications[ this.parameter_choice_index ];
        
    }
    
    
    // Method to obtain the array of the parameters required to build
    // the CRYSTALS-Kyber Public-Key Cryptosystem's cryptographic engine.
    public Object[] get_engine_parameters() {
        
        // Return of the array of the parameters required to build
        // the CRYSTALS-Kyber Public-Key Cryptosystem's cryptographic engine.
        return engine_parameters[ this.parameter_choice_index ];
        
    }
    
    
    // Method to initialize the general parameters to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public void initialize_parameters() {
        
        // If the parameter choice index for
        // the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // corresponds to the 'Kyber512' Security Parameter Set.
        if( this.parameter_choice_index == 0 ) {
            
            // Setting the 'Kyber512' Security Parameter Set as
            // the parameter choice for the general parameters
            // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            this.parameters = KyberParameters.kyber512;
            
        }
        // If the parameter choice index for
        // the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // corresponds to the 'Kyber768' Security Parameter Set.
        else if( this.parameter_choice_index == 1 ) {
            
            // Setting the 'Kyber768' Security Parameter Set as
            // the parameter choice for the general parameters
            // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            this.parameters = KyberParameters.kyber768;
            
        }
        // If the parameter choice index for
        // the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // corresponds to the 'Kyber1024' Security Parameter Set.
        else if( this.parameter_choice_index == 2 ) {
            
            // Setting the 'Kyber1024' Security Parameter Set as
            // the parameter choice for the general parameters
            // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            this.parameters = KyberParameters.kyber1024;
            
        }
        
    }
    
    
    // Method to obtain the general parameters to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public KyberParameters get_parameters() {
        
        // Return of the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.parameters;
        
    }
    
    
    // Method to obtain the size of the session key, in terms of bytes,
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_session_key_bytes_size() {
        
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not null (i.e., already initialized).
        if( this.parameters != null ) {
            
            // Return of the size of the session key, in terms of bytes,
            // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return this.get_parameters().getSessionKeySize();
            
        }
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is null (i.e., not initialized yet).
        else {
            
            // Return of an invalid size of the session key, in terms of bytes,
            // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return -1;

        }
        
    }
    
    
    // Method to obtain the size of the session key, in terms of bits,
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_session_key_bits_size() {
        
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not null (i.e., already initialized).
        if( this.parameters != null ) {
            
            // Return of the size of the session key, in terms of bits,
            // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return ( this.get_session_key_bytes_size() * BYTE_SIZE_IN_BITS );
            
        }
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is null (i.e., not initialized yet).
        else {
            
            // Return of an invalid size of the session key, in terms of bits,
            // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return -1;

        }
        
    }
    
    
    // Method to obtain the cryptographic engine to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public KyberEngine get_engine() {
        
        // Retrieval of the parameters required to build
        // the CRYSTALS-Kyber Public-Key Cryptosystem's cryptographic engine.
        Object[] kyber_engine_parameters = this.get_engine_parameters();

        // Return of the cryptographic engine with
        // the 'Kyber512' Security Parameter Set as
        // the parameter choice for the general parameters
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return new KyberEngine( (int) kyber_engine_parameters[0],
                                (boolean) kyber_engine_parameters[1] );
        
    }
        
    
    // Method to obtain the maximum degree n of
    // the polynomials used in the matrix A representing
    // the lattice algebraic structure to be adopted for
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_max_degree_polynomials_n_from_engine() {
        
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not null (i.e., already initialized).
        if( this.parameters != null ) {

            // Return of the maximum degree n of
            // the polynomials used in the matrix A representing
            // the lattice algebraic structure to be adopted for
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return this.get_engine().KyberN;

        }
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is null (i.e., not initialized yet).
        else {

            // Return of an invalid maximum degree n of
            // the polynomials used in the matrix A representing
            // the lattice algebraic structure to be adopted for
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return -1;
        
        }
    
    }
        
    
    // Method to obtain the number of polynomials k
    // per vector used in the matrix A representing
    // the lattice algebraic structure, representing as well,
    // the size factor for that same lattice algebraic structure,
    // and thus, being the main security parameter in terms of scaling
    // the security strength of the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_num_polynomials_per_vector_k_from_engine() {
        
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not null (i.e., already initialized).
        if( this.parameters != null ) {
         
            // Return of the number of polynomials k
            // per vector used in the matrix A representing
            // the lattice algebraic structure, representing as well,
            // the size factor for that same lattice algebraic structure,
            // and thus, being the main security parameter in terms of scaling
            // the security strength of the CRYSTALS-Kyber Public-Key Cryptosystem.
            return this.get_engine().getKyberK();
            
        }
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is null (i.e., not initialized yet).
        else {
            
            // Return of an invalid number of polynomials k
            // per vector used in the matrix A representing
            // the lattice algebraic structure, representing as well,
            // the size factor for that same lattice algebraic structure,
            // and thus, being the main security parameter in terms of scaling
            // the security strength of the CRYSTALS-Kyber Public-Key Cryptosystem.
            return -1;
            
        }
        
    }
    
    
    // Method to obtain the numeric modulus q for
    // the polynomials used in the matrix A representing
    // the lattice algebraic structure to be adopted for
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_numeric_modulus_q_from_engine() {
    
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not null (i.e., already initialized).
        if( this.parameters != null ) {
         
            // Return of the numeric modulus q for
            // the polynomials used in the matrix A representing
            // the lattice algebraic structure to be adopted for
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return this.get_engine().KyberQ;
            
        }
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is null (i.e., not initialized yet).
        else {
            
            // Return of an invalid numeric modulus q for
            // the polynomials used in the matrix A representing
            // the lattice algebraic structure to be adopted for
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return -1;
            
        }
        
    }
    
    
    // Method to obtain the compression factor d for
    // the size of the asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_asymmetric_keys_compression_factor_d_from_engine() {
    
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not null (i.e., already initialized).
        if( this.parameters != null ) {
         
            // Return of the compression factor d for
            // the size of the asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return (int) ( Math.log( (double) this.get_numeric_modulus_q_from_engine() ) / Math.log( (double) 2 ) );
            
        }
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is null (i.e., not initialized yet).
        else {
            
            // Return of an invalid compression factor d for
            // the size of the asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return -1;
            
        }
        
    }
    
    
    // Method to obtain the size of the symmetric
    // hashes and (pseudo) random seeds, in terms of bytes,
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_symmetric_hashes_and_random_seeds_bytes_size_from_engine() {
    
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not null (i.e., already initialized).
        if( this.parameters != null ) {
         
            // Return of the size of the symmetric
            // hashes and (pseudo) random seeds, in terms of bytes,
            // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return this.get_engine().KyberSymBytes;
            
        }
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is null (i.e., not initialized yet).
        else {
            
            // Return of an invalid size of the symmetric
            // hashes and (pseudo) random seeds, in terms of bytes,
            // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return -1;
            
        }
        
    }
    
    
    // Method to obtain the size of the symmetric
    // hashes and (pseudo) random seeds, in terms of bits,
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_symmetric_hashes_and_random_seeds_bits_size_from_engine() {
    
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not null (i.e., already initialized).
        if( this.parameters != null ) {
         
            // Return of the size of the symmetric
            // hashes and (pseudo) random seeds, in terms of bits,
            // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return ( this.get_symmetric_hashes_and_random_seeds_bits_size_from_engine() * BYTE_SIZE_IN_BITS );
            
        }
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is null (i.e., not initialized yet).
        else {
            
            // Return of an invalid size of the symmetric
            // hashes and (pseudo) random seeds, in terms of bits,
            // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return -1;
            
        }
        
    }
    
    
    // Method to obtain the size of the symmetric
    // shared secret, in terms of bytes, to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_symmetric_shared_secret_bytes_size_from_engine() {
    
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not null (i.e., already initialized).
        if( this.parameters != null ) {
         
            // Return of the size of the symmetric
            // shared secret, in terms of bytes, to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return this.get_engine().KyberSharedSecretBytes;
            
        }
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is null (i.e., not initialized yet).
        else {
            
            // Return of an invalid size of the symmetric
            // shared secret, in terms of bytes, to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return -1;
            
        }
        
    }
    
    
    // Method to obtain the size of the symmetric
    // shared secret, in terms of bits, to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_symmetric_shared_secret_bits_size_from_engine() {
    
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not null (i.e., already initialized).
        if( this.parameters != null ) {
         
            // Return of the size of the symmetric
            // shared secret, in terms of bits, to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return ( this.get_symmetric_shared_secret_bytes_size_from_engine() * BYTE_SIZE_IN_BITS );
            
        }
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is null (i.e., not initialized yet).
        else {
            
            // Return of an invalid size of the symmetric
            // shared secret, in terms of bits, to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return -1;
            
        }
        
    }
    
    
    // Method to obtain the numeric noise factor eta_1
    // used in the secret random coins vector r
    // extracted from a Binomial Probability Distribution,
    // dependent on the public key k_pub, and also
    // used for the error vector e and secret vector s,
    // regarding the polynomials representing a lattice
    // algebraic structure for the mathematical equation
    // A x s + e = t), representing a lattice-based problem in
    // the form of a MLWE (Module-Learning With Errors), to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_noise_factor_eta_1_from_engine() {
    
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not null (i.e., already initialized).
        if( this.parameters != null ) {
         
            // Return of the numeric noise factor eta_1
            // used in the secret random coins vector r
            // extracted from a Binomial Probability Distribution,
            // dependent on the public key k_pub, and also
            // used for the error vector e and secret vector s,
            // regarding the polynomials representing a lattice
            // algebraic structure for the mathematical equation
            // A x s + e = t), representing a lattice-based problem in
            // the form of a MLWE (Module-Learning With Errors), to be
            // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return this.get_engine().getKyberEta1();
            
        }
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is null (i.e., not initialized yet).
        else {
            
            // Return of an invalid numeric noise factor eta_1
            // used in the secret random coins vector r
            // extracted from a Binomial Probability Distribution,
            // dependent on the public key k_pub, and also
            // used for the error vector e and secret vector s,
            // regarding the polynomials representing a lattice
            // algebraic structure for the mathematical equation
            // A x s + e = t), representing a lattice-based problem in
            // the form of a MLWE (Module-Learning With Errors), to be
            // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return -1;
            
        }
        
    }
    
    
    // Method to obtain the numeric noise factor eta_2
    // used for the random error vectors e_1 and e_2 extracted
    // from a (Centered) Binomial Probability Distribution,
    // applied during the asymmetric encryption and generation of
    // the KEM (Key Encapsulation Method/Mechanism) procedures
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_noise_factor_eta_2_from_engine() {
    
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not null (i.e., already initialized).
        if( this.parameters != null ) {
         
            // Return of the numeric noise factor eta_2
            // used for the random error vectors e_1 and e_2 extracted
            // from a (Centered) Binomial Probability Distribution,
            // applied during the asymmetric encryption and generation of
            // the KEM (Key Encapsulation Method/Mechanism) procedures
            // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return this.get_engine().getKyberEta2();
            
        }
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is null (i.e., not initialized yet).
        else {
            
            // Return of an invalid numeric noise factor eta_2
            // used for the random error vectors e_1 and e_2 extracted
            // from a (Centered) Binomial Probability Distribution,
            // applied during the asymmetric encryption and generation of
            // the KEM (Key Encapsulation Method/Mechanism) procedures
            // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return -1;
            
        }
        
    }
    
    
    // Method to obtain the size in terms of bytes of
    // a polynomial compressed in an array of bytes, to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_polynomial_compressed_bytes_size_from_engine() {
    
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not null (i.e., already initialized).
        if( this.parameters != null ) {
         
            // Return of the size in terms of bytes of
            // a polynomial compressed in an array of bytes, to be
            // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return this.get_engine().getKyberPolyCompressedBytes();
            
        }
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is null (i.e., not initialized yet).
        else {
            
            // Return of an invalid size in terms of bytes of
            // a polynomial compressed in an array of bytes, to be
            // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return -1;
            
        }
        
    }
    
    
    // Method to obtain the size in terms of bits of
    // a polynomial compressed in an array of bytes, to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_polynomial_compressed_bits_size_from_engine() {
    
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not null (i.e., already initialized).
        if( this.parameters != null ) {
         
            // Return of the size in terms of bits of
            // a polynomial compressed in an array of bytes, to be
            // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return ( this.get_polynomial_compressed_bytes_size_from_engine() * BYTE_SIZE_IN_BITS );
            
        }
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is null (i.e., not initialized yet).
        else {
            
            // Return of an invalid size in terms of bits of
            // a polynomial compressed in an array of bytes, to be
            // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return -1;
            
        }
        
    }
    
    
    // Method to obtain the size in terms of bytes of
    // a polynomials vector compressed in a matrix of bytes,
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_polynomials_vector_compressed_bytes_size_from_engine() {
    
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not null (i.e., already initialized).
        if( this.parameters != null ) {
         
            // Return of the size in terms of bytes of
            // a polynomials vector compressed in a matrix of bytes,
            // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return this.get_engine().getKyberPolyVecCompressedBytes();
            
        }
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is null (i.e., not initialized yet).
        else {
            
            // Return of an invalid size in terms of bytes of
            // a polynomials vector compressed in a matrix of bytes,
            // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return -1;
            
        }
        
    }
    
    
    // Method to obtain the size in terms of bits of
    // a polynomials vector compressed in a matrix of bytes,
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_polynomials_vector_compressed_bits_size_from_engine() {
    
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not null (i.e., already initialized).
        if( this.parameters != null ) {
         
            // Return of the size in terms of bits of
            // a polynomials vector compressed in a matrix of bytes,
            // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return ( this.get_polynomials_vector_compressed_bytes_size_from_engine() * BYTE_SIZE_IN_BITS );
            
        }
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is null (i.e., not initialized yet).
        else {
            
            // Return of an invalid size in terms of bits of
            // a polynomial compressed in an array of bytes, to be
            // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return -1;
            
        }
        
    }
    
    
    // Method to obtain the size in terms of bytes of
    // the exchaged session symmetric key in bytes, to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_session_key_bytes_size_from_engine() {
    
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not null (i.e., already initialized).
        if( this.parameters != null ) {
         
            // Return of the size in terms of bytes of
            // the exchaged session symmetric key in bytes, to be
            // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return this.get_engine().getSessionKeyLength();
            
        }
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is null (i.e., not initialized yet).
        else {
            
            // Return of an invalid size in terms of bytes of
            // the exchaged session symmetric key in bytes, to be
            // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return -1;
            
        }
        
    }
    
    
    // Method to obtain the size in terms of bits of
    // the exchaged session symmetric key in bytes, to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_session_key_bits_size_from_engine() {
    
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not null (i.e., already initialized).
        if( this.parameters != null ) {
         
            // Return of the size in terms of bits of
            // the exchaged session symmetric key in bytes, to be
            // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return ( this.get_session_key_bytes_size_from_engine() * BYTE_SIZE_IN_BITS );
            
        }
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is null (i.e., not initialized yet).
        else {
            
            // Return of an invalid size in terms of bits of
            // the exchaged session symmetric key in bytes, to be
            // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return -1;
            
        }
        
    }
    
    
    // Method to obtain the Java's Secure Random to act as
    // Pseudo-Random Generator (PRG) for the key pair generation to
    // be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public SecureRandom get_secure_pseudo_random_generator_key_pair_generation() {
            
        // Return of the Java's Secure Random to act as
        // Pseudo-Random Generator (PRG) for the key pair generation to
        // be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.secure_pseudo_random_generator_key_pair_generation;
    
    }
    
    
    // Method to initialize the key generation parameters to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public void initialize_key_generation_parameters() {
        
        // If the the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem, are not null.
        if( this.parameters != null ) {
        
            // Initialization of the key generation parameters to be
            // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            this.key_generation_parameters = 
                new KyberKeyGenerationParameters
                    ( this.secure_pseudo_random_generator_key_pair_generation,
                      this.parameters );
        
        }
        
    }
    
    
    // Method to obtain the key generation parameters to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public KyberKeyGenerationParameters get_key_generation_parameters() {
        
        // Return of the key generation parameters to be
        // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.key_generation_parameters;
        
    }
    
    
    // Method to initializes the generation of
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem. 
    public void initializes_asymmetric_key_pair_generation() {
       
        // Creation of the generator of the pair of asymmetric keys
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem. 
        KyberKeyPairGenerator crystals_kyber_key_pair_generator = 
            new KyberKeyPairGenerator();

        // Initialization of the generator of the pair of asymmetric keys
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
        // using the Security Parameters set defined before.
        crystals_kyber_key_pair_generator
            .init( this.key_generation_parameters );

        // Generation of the pair of asymmetric keys to be
        // used by the CRYSTALS-Kyber Public-Key Cryptosystem. 
        this.asymmetric_key_pair = 
            crystals_kyber_key_pair_generator.generateKeyPair();
            
    }
    
    
    // Method to obtain the pair of asymmetric keys to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public AsymmetricCipherKeyPair get_asymmetric_key_pair() {
        
        // Return of the pair of asymmetric keys to be
        // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.asymmetric_key_pair;
        
    }
    
    
    // Method to obtain the public key from
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public AsymmetricKeyParameter get_public_key() {
        
        // If the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not defined by a null object.
        if( this.asymmetric_key_pair != null ) {
            
            // Return of the public key from
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return this.asymmetric_key_pair.getPublic();
            
        }
        // If the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is defined by a null object.
        else {
           
            // Return of a null object.
            return null;
            
        }
        
    }
    
    
    // Method to obtain the public key parameters of
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public KyberPublicKeyParameters get_public_key_parameters() {
        
        // If the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not defined by a null object.
        if( this.asymmetric_key_pair != null ) {
            
            // Return of the public key parameters of
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return ( ( KyberPublicKeyParameters ) this.get_public_key() );
            
        }
        // If the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is defined by a null object.
        else {
           
            // Return of a null object.
            return null;
            
        }
        
    }
    
    
    // Method to obtain the target solution t
    // (i.e. the target solution vector for
    // the mathematical equation A x s + e = t),
    // encoded (and compressed) in bytes,
    // representing a lattice-based problem in
    // the form of a LWE (Learning With Errors),
    // retrieved from the public key parameters of
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public byte[] get_target_solution_t_from_public_key_parameters() {
        
        // If the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not defined by a null object.
        if( this.asymmetric_key_pair != null ) {
            
            // Return of the target solution t
            // (i.e. the target solution vector for
            // the mathematical equation A x s + e = t),
            // encoded (and compressed) in bytes,
            // representing a lattice-based problem in
            // the form of a LWE (Learning With Errors),
            // retrieved from the public key parameters of
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return this.get_public_key_parameters().getT();
            
        }
        // If the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is defined by a null object.
        else {
           
            // Return of a null object.
            return null;
            
        }
        
    }
    
    
    // Method to obtain the size in terms of
    // bytes of the target solution t
    // (i.e. the target solution vector for
    // the mathematical equation A x s + e = t),
    // encoded (and compressed) in bytes,
    // representing a lattice-based problem in
    // the form of a LWE (Learning With Errors),
    // retrieved from the public key parameters of
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_target_solution_t_size_bytes_from_public_key_parameters() {
        
        // Return of the size in terms of
        // bytes of the target solution t
        // (i.e. the target solution vector for
        // the mathematical equation A x s + e = t),
        // encoded (and compressed) in bytes,
        // representing a lattice-based problem in
        // the form of a LWE (Learning With Errors),
        // retrieved from the public key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.get_target_solution_t_from_public_key_parameters().length;
        
    }
    
    
    // Method to obtain the size in terms of
    // bits of the target solution t
    // (i.e. the target solution vector for
    // the mathematical equation A x s + e = t),
    // encoded (and compressed) in bytes,
    // representing a lattice-based problem in
    // the form of a LWE (Learning With Errors),
    // retrieved from the public key parameters of
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_target_solution_t_size_bits_from_public_key_parameters() {
        
        // Return of the size in terms of
        // bits of the target solution t
        // (i.e. the target solution vector for
        // the mathematical equation A x s + e = t),
        // encoded (and compressed) in bytes,
        // representing a lattice-based problem in
        // the form of a LWE (Learning With Errors),
        // retrieved from the public key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem.
        return ( this.get_target_solution_t_size_bytes_from_public_key_parameters() * BYTE_SIZE_IN_BITS );
        
    }
    
    
    // Method to obtain the (pseudo) random seed ρ (rho)
    // (i.e. the random seed from will be expanded and
    // obtained the uniformly-random matrix A consisting of
    // polynomials representing a lattice algebraic structure for
    // the mathematical equation A x s + e = t), encoded in bytes,
    // representing a lattice-based problem in the form of a
    // MLWE (Module-Learning With Errors), retrieved from
    // the public key parameters of the pair of asymmetric keys
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public byte[] get_random_seed_rho_from_public_key_parameters() {
        
        // If the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not defined by a null object.
        if( this.asymmetric_key_pair != null ) {
            
            // Return of the (pseudo) random seed ρ (rho)
            // (i.e. the random seed from will be expanded and
            // obtained the uniformly-random matrix A consisting of
            // polynomials representing a lattice algebraic structure for
            // the mathematical equation A x s + e = t), encoded in bytes,
            // representing a lattice-based problem in the form of a
            // MLWE (Module-Learning With Errors), retrieved from
            // the public key parameters of the pair of asymmetric keys
            // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return this.get_public_key_parameters().getRho();
            
        }
        // If the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is defined by a null object.
        else {
           
            // Return of a null object.
            return null;
            
        }
        
    }
    
    
    // Method to obtain the size in terms of
    // bytes of the (pseudo) random seed ρ (rho)
    // (i.e. the random seed from will be expanded and
    // obtained the uniformly-random matrix A consisting of
    // polynomials representing a lattice algebraic structure for
    // the mathematical equation A x s + e = t), encoded in bytes,
    // representing a lattice-based problem in the form of a
    // MLWE (Module-Learning With Errors), retrieved from
    // the public key parameters of the pair of asymmetric keys
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_random_seed_rho_size_bytes_from_public_key_parameters() {
        
        // Return of the size in terms of
        // bytes of the (pseudo) random seed ρ (rho)
        // (i.e. the random seed from will be expanded and
        // obtained the uniformly-random matrix A consisting of
        // polynomials representing a lattice algebraic structure for
        // the mathematical equation A x s + e = t), encoded in bytes,
        // representing a lattice-based problem in the form of a
        // MLWE (Module-Learning With Errors), retrieved from
        // the public key parameters of the pair of asymmetric keys
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.get_random_seed_rho_from_public_key_parameters().length;
        
    }
    
    
    // Method to obtain the size in terms of
    // bits of the (pseudo) random seed ρ (rho)
    // (i.e. the random seed from will be expanded and
    // obtained the uniformly-random matrix A consisting of
    // polynomials representing a lattice algebraic structure for
    // the mathematical equation A x s + e = t), encoded in bytes,
    // representing a lattice-based problem in the form of a
    // MLWE (Module-Learning With Errors), retrieved from
    // the public key parameters of the pair of asymmetric keys
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_random_seed_rho_size_bits_from_public_key_parameters() {
        
        // Return of the size in terms of
        // bits of the (pseudo) random seed ρ (rho)
        // (i.e. the random seed from will be expanded and
        // obtained the uniformly-random matrix A consisting of
        // polynomials representing a lattice algebraic structure for
        // the mathematical equation A x s + e = t), encoded in bytes,
        // representing a lattice-based problem in the form of a
        // MLWE (Module-Learning With Errors), retrieved from
        // the public key parameters of the pair of asymmetric keys
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return ( this.get_random_seed_rho_size_bytes_from_public_key_parameters() * BYTE_SIZE_IN_BITS );
        
    }
    
    
    // Method to obtain the public key from
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem,
    // being encoded as an array of bytes.
    public byte[] get_public_key_encoded_bytes() {
        
        // Retrieval of the public key from
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // being encoded as an array of bytes.
        byte[] public_key_encoded_bytes = 
             ( ( KyberPublicKeyParameters ) this.get_public_key() ).getEncoded();
        
        // Return of the public key from
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // being encoded as an array of bytes.
        return public_key_encoded_bytes;
        
    }
    
    
    // Method to obtain the size of the public key
    // from the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem,
    // as being encoded as an array of bytes.
    public int get_public_key_size_bytes() {
        
        // Return of the size of the public key
        // from the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // as being encoded as an array of bytes.
        return this.get_public_key_encoded_bytes().length;
        
    }
    
    
    // Method to obtain the size of the public key
    // from the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem,
    // as being encoded as an array of bits.
    public int get_public_key_size_bits() {
        
        // Return of the size of the public key
        // from the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // as being encoded as an array of bits.
        return ( this.get_public_key_size_bytes() * BYTE_SIZE_IN_BITS );
        
    }
    
    
    // Method to obtain the payload of the public key
    // from the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem,
    // as a proper object of public key of the cryptosystem,
    // using the associated Security Parameters.
    public KyberPublicKeyParameters get_public_key_payload_from_parameters() {
        
        // Definition of the payload of the public key
        // from the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // as a proper object of public key of the cryptosystem.
        KyberPublicKeyParameters public_key_payload;
        
        // Retrieval of the cipher parameters from
        // the key generation parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem.
        KyberParameters crystals_kyber_cryptosystem_parameters = 
            this.get_key_generation_parameters().getParameters();
        
        
        // Call of the required methods to
        // obtain all the values, encoded in bytes,
        // retrieved from the public key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // which will be used to build public key payload.
        
        // Retrieval of the target solution t
        // (i.e. the target solution vector for
        // the mathematical equation A x s + e = t),
        // encoded (and compressed) in bytes.
        byte[] target_solution_t = 
            this.get_target_solution_t_from_public_key_parameters();
        
        // Retrieval of the (pseudo) random seed ρ (rho)
        // (i.e. the random seed from will be expanded and
        // obtained the uniformly-random matrix A consisting of
        // polynomials representing a lattice algebraic structure for
        // the mathematical equation A x s + e = t), encoded in bytes.
        byte[] random_seed_rho = 
            this.get_random_seed_rho_from_public_key_parameters();
        
        
        // Creation of the payload of the public key
        // from the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // as a proper object of public key of the cryptosystem.
        public_key_payload =
            new KyberPublicKeyParameters( crystals_kyber_cryptosystem_parameters,
                                          target_solution_t,
                                          random_seed_rho );
        
        
        // Return of the payload of the public key
        // from the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // as a proper object of public key of the cryptosystem.
        return public_key_payload;
        
    }
    
    
    // Method to obtain the payload of the public key
    // from the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem,
    // as a proper object of public key of the cryptosystem,
    // using the associated encoding in an array of bytes.
    public KyberPublicKeyParameters get_public_key_payload_from_encoding() {
        
        // Definition of the payload of the public key
        // from the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // as a proper object of public key of the cryptosystem.
        KyberPublicKeyParameters public_key_payload;
        
        // Retrieval of the cipher parameters from
        // the key generation parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem.
        KyberParameters crystals_kyber_cryptosystem_parameters = 
            this.get_key_generation_parameters().getParameters();
        
        
        // Creation of the payload of the public key
        // from the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // as a proper object of public key of the cryptosystem.
        public_key_payload =
            new KyberPublicKeyParameters( crystals_kyber_cryptosystem_parameters,
                                          this.get_public_key_encoded_bytes() );
        
        
        // Return of the payload of the public key
        // from the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // as a proper object of public key of the cryptosystem.
        return public_key_payload;
        
    }
    
    // Method to obtain the payload of the public key
    // from the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem,
    // as a proper object of public key of the cryptosystem.
    public KyberPublicKeyParameters get_public_key_payload() {
    
        // Return of the payload of the public key
        // from the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // as a proper object of public key of the cryptosystem.
        return this.get_public_key_payload_from_parameters();
        
    }
    
    // Method to obtain the private key from
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public AsymmetricKeyParameter get_private_key() {
        
        // If the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not defined by a null object.
        if( this.asymmetric_key_pair != null ) {
            
            // Return of the private key from
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return this.asymmetric_key_pair.getPrivate();
            
        }
        // If the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is defined by a null object.
        else {
           
            // Return of a null object.
            return null;
            
        }
        
    }
    
    
    // Method to obtain the private key parameters of
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public KyberPrivateKeyParameters get_private_key_parameters() {
        
        // If the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not defined by a null object.
        if( this.asymmetric_key_pair != null ) {
            
            // Return of the private key parameters of
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return ( ( KyberPrivateKeyParameters ) this.get_private_key() );
            
        }
        // If the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is defined by a null object.
        else {
           
            // Return of a null object.
            return null;
            
        }
        
    }
    
    
    // Method to obtain the random secret s
    // (i.e. the secret vector in the lattice
    // algebraic structure), encoded in bytes,
    // with the IND-CPA (INDistinguishability
    // under Chosen Plaintext Attack) property,
    // retrieved from the private key parameters of
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public byte[] get_random_secret_s_from_private_key_parameters() {
        
        // If the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not defined by a null object.
        if( this.asymmetric_key_pair != null ) {
            
            // Return of the random secret s
            // (i.e. the secret vector in the lattice
            // algebraic structure), encoded in bytes,
            // with the IND-CPA (INDistinguishability
            // under Chosen Plaintext Attack) property,
            // retrieved from the private key parameters of
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return this.get_private_key_parameters().getS();
            
        }
        // If the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is defined by a null object.
        else {
           
            // Return of a null object.
            return null;
            
        }
        
    }
    
    
    // Method to obtain the size in
    // terms of bytes of the random secret s
    // (i.e. the secret vector in the lattice
    // algebraic structure), encoded in bytes,
    // with the IND-CPA (INDistinguishability
    // under Chosen Plaintext Attack) property,
    // retrieved from the private key parameters of
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_random_secret_s_size_bytes_from_private_key_parameters() {
        
        // Return of the size in terms of
        // bytes of the random secret s
        // (i.e. the secret vector in the lattice
        // algebraic structure), encoded in bytes,
        // with the IND-CPA (INDistinguishability
        // under Chosen Plaintext Attack) property,
        // retrieved from the private key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.get_random_secret_s_from_private_key_parameters().length;
        
    }
    
    
    // Method to obtain the size in
    // terms of bits of the random secret s
    // (i.e. the secret vector in the lattice
    // algebraic structure), encoded in bytes,
    // with the IND-CPA (INDistinguishability
    // under Chosen Plaintext Attack) property,
    // retrieved from the private key parameters of
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_random_secret_s_size_bits_from_private_key_parameters() {
        
        // Return of the size in terms of
        // bytes of the random secret s
        // (i.e. the secret vector in the lattice
        // algebraic structure), encoded in bytes,
        // with the IND-CPA (INDistinguishability
        // under Chosen Plaintext Attack) property,
        // retrieved from the private key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem.
        return ( this.get_random_secret_s_size_bytes_from_private_key_parameters() * BYTE_SIZE_IN_BITS );
        
    }
    
    
    // Method to obtain the hashed public key,
    // denoted as H(pub_key), encoded in bytes,
    // with the IND-CPA (INDistinguishability
    // under Chosen Plaintext Attack) property,
    // retrieved from the private key parameters of
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public byte[] get_hashed_public_key_h_pub_k_from_private_key_parameters() {
        
        // If the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not defined by a null object.
        if( this.asymmetric_key_pair != null ) {
            
            // Return of the hashed public key,
            // denoted as H(pub_key), encoded in bytes,
            // with the IND-CPA (INDistinguishability
            // under Chosen Plaintext Attack) property,
            // retrieved from the private key parameters of
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return this.get_private_key_parameters().getHPK();
            
        }
        // If the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is defined by a null object.
        else {
           
            // Return of a null object.
            return null;
            
        }
        
    }
    
    
    // Method to obtain the size in
    // terms of bytes of the hashed public key,
    // denoted as H(pub_key), encoded in bytes,
    // with the IND-CPA (INDistinguishability
    // under Chosen Plaintext Attack) property,
    // retrieved from the private key parameters of
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_hashed_public_key_h_pub_k_size_bytes_from_private_key_parameters() {
        
        // Return of the size in
        // terms of bytes of the hashed public key,
        // denoted as H(pub_key), encoded in bytes,
        // with the IND-CPA (INDistinguishability
        // under Chosen Plaintext Attack) property,
        // retrieved from the private key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.get_hashed_public_key_h_pub_k_from_private_key_parameters().length;
        
    }
    
    
    // Method to obtain the size in
    // terms of bits of the hashed public key,
    // denoted as H(pub_key), encoded in bytes,
    // with the IND-CPA (INDistinguishability
    // under Chosen Plaintext Attack) property,
    // retrieved from the private key parameters of
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_hashed_public_key_h_pub_k_size_bits_from_private_key_parameters() {
        
        // Return of the size in
        // terms of bits of the hashed public key,
        // denoted as H(pub_key), encoded in bytes,
        // with the IND-CPA (INDistinguishability
        // under Chosen Plaintext Attack) property,
        // retrieved from the private key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem.
        return ( this.get_hashed_public_key_h_pub_k_size_bytes_from_private_key_parameters() * BYTE_SIZE_IN_BITS );
        
    }
    
    
    // Method to obtain the random secret nonce z
    // (i.e., to avoid practical fault attacks),
    // encoded in bytes, with the IND-CPA
    // (INDistinguishability under Chosen Plaintext
    // Attack) property for the Asymmetric Encryption and
    // with the (INDistinguishability under Chosen Ciphertext
    // Attack) property for the KEM (Key Encapsulation
    // Method/Mechanism), retrieved from the private key
    // parameters of the pair of asymmetric keys to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public byte[] get_random_secret_nonce_z_from_private_key_parameters() {
        
        // If the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not defined by a null object.
        if( this.asymmetric_key_pair != null ) {
            
            // Return of the random secret nonce z
            // (i.e., to avoid practical fault attacks),
            // encoded in bytes, with the IND-CPA
            // (INDistinguishability under Chosen Plaintext
            // Attack) property for the Asymmetric Encryption and
            // with the (INDistinguishability under Chosen Ciphertext
            // Attack) property for the KEM (Key Encapsulation
            // Method/Mechanism), retrieved from the private key
            // parameters of the pair of asymmetric keys to be
            // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return this.get_private_key_parameters().getNonce();
            
        }
        // If the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is defined by a null object.
        else {
           
            // Return of a null object.
            return null;
            
        }
        
    }
    
    
    // Method to obtain the size in terms of
    // bytes of the random secret nonce z
    // (i.e., to avoid practical fault attacks),
    // encoded in bytes, with the IND-CPA
    // (INDistinguishability under Chosen Plaintext
    // Attack) property for the Asymmetric Encryption and
    // with the (INDistinguishability under Chosen Ciphertext
    // Attack) property for the KEM (Key Encapsulation
    // Method/Mechanism), retrieved from the private key
    // parameters of the pair of asymmetric keys to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int random_secret_nonce_z_size_bytes_from_private_key_parameters() {
        
        // Return of the size in terms of
        // bytes of the random secret nonce z
        // (i.e., to avoid practical fault attacks),
        // encoded in bytes, with the IND-CPA
        // (INDistinguishability under Chosen Plaintext
        // Attack) property for the Asymmetric Encryption and
        // with the (INDistinguishability under Chosen Ciphertext
        // Attack) property for the KEM (Key Encapsulation
        // Method/Mechanism), retrieved from the private key
        // parameters of the pair of asymmetric keys to be
        // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.get_random_secret_nonce_z_from_private_key_parameters().length;
        
    }
    
    
    // Method to obtain the size in terms of
    // bits of the random secret nonce z
    // (i.e., to avoid practical fault attacks),
    // encoded in bytes, with the IND-CPA
    // (INDistinguishability under Chosen Plaintext
    // Attack) property for the Asymmetric Encryption and
    // with the (INDistinguishability under Chosen Ciphertext
    // Attack) property for the KEM (Key Encapsulation
    // Method/Mechanism), retrieved from the private key
    // parameters of the pair of asymmetric keys to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int random_secret_nonce_z_size_bits_from_private_key_parameters() {
        
        // Return of the size in terms of
        // bits of the random secret nonce z
        // (i.e., to avoid practical fault attacks),
        // encoded in bytes, with the IND-CPA
        // (INDistinguishability under Chosen Plaintext
        // Attack) property for the Asymmetric Encryption and
        // with the (INDistinguishability under Chosen Ciphertext
        // Attack) property for the KEM (Key Encapsulation
        // Method/Mechanism), retrieved from the private key
        // parameters of the pair of asymmetric keys to be
        // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return ( this.random_secret_nonce_z_size_bytes_from_private_key_parameters() * BYTE_SIZE_IN_BITS );
        
    }
    
    
    // Method to obtain the private key from
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem,
    // being encoded as an array of bytes.
    public byte[] get_private_key_encoded_bytes() {
        
        // Retrieval of the private key from
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // being encoded as an array of bytes.
        byte[] private_key_encoded_bytes = 
             ( ( KyberPrivateKeyParameters ) this.get_private_key() ).getEncoded();
        
        // Return of the private key from
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // being encoded as an array of bytes.
        return private_key_encoded_bytes;
        
    }
    
    
    // Method to obtain the size of the private key
    // from the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem,
    // as being encoded as an array of bytes.
    public int get_private_key_size_bytes() {
        
        // Return of the size of the private key
        // from the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // as being encoded as an array of bytes.
        return this.get_private_key_encoded_bytes().length;
        
    }
    
    
    // Method to obtain the size of the private key
    // from the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem,
    // as being encoded as an array of bits.
    public int get_private_key_size_bits() {
        
        // Return of the size of the private key
        // from the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // as being encoded as an array of bits.
        return ( this.get_private_key_size_bytes() * BYTE_SIZE_IN_BITS );
        
    }
    
    
    // Method to obtain the payload of the private key
    // from the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem,
    // as a proper object of private key of the cryptosystem.
    public KyberPrivateKeyParameters get_private_key_payload() {
        
        // Definition of the payload of the private key
        // from the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // as a proper object of private key of the cryptosystem.
        KyberPrivateKeyParameters private_key_payload;
        
        // Retrieval of the cipher parameters from
        // the key generation parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem.
        KyberParameters crystals_kyber_cryptosystem_parameters = 
            this.get_key_generation_parameters().getParameters();
        
        
        // Call of the required methods to
        // obtain all the values, encoded in bytes,
        // retrieved from the public key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // which will be used to build private key payload.
        
        // Retrieval of the random secret s
        // (i.e. the secret vector in the lattice
        // algebraic structure), encoded in bytes.
        byte[] random_secret_s = 
            this.get_random_secret_s_from_private_key_parameters();
        
        // Retrieval of the hashed public key,
        // denoted as H(pub_key), encoded in bytes.
        byte[] hashed_public_key_h_pub_k = 
            this.get_hashed_public_key_h_pub_k_from_private_key_parameters();
        
        // Retrieval of the random secret
        // nonce z (i.e., to avoid practical
        // fault attacks), encoded in bytes.
        byte[] random_secret_nonce_z = 
            this.get_random_secret_nonce_z_from_private_key_parameters();
        
        
        // Retrieval of the target solution t
        // (i.e. the target solution vector for
        // the mathematical equation A x s + e = t),
        // encoded (and compressed) in bytes.
        byte[] target_solution_t = 
            this.get_target_solution_t_from_public_key_parameters();
        
        // Retrieval of the (pseudo) random seed ρ (rho)
        // (i.e. the random seed from will be expanded and
        // obtained the uniformly-random matrix A consisting of
        // polynomials representing a lattice algebraic structure for
        // the mathematical equation A x s + e = t), encoded in bytes.
        byte[] random_seed_rho = 
            this.get_random_seed_rho_from_public_key_parameters();
        
        
        // Creation of the payload of the private key
        // from the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // as a proper object of private key of the cryptosystem.
        private_key_payload =
            new KyberPrivateKeyParameters( crystals_kyber_cryptosystem_parameters,
                                           random_secret_s,
                                           hashed_public_key_h_pub_k,
                                           random_secret_nonce_z,
                                           target_solution_t,
                                           random_seed_rho );
        
        
        // Return of the payload of the private key
        // from the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // as a proper object of private key of the cryptosystem.
        return private_key_payload;
        
    }
    
    
    // Method to obtain the Java's Secure Random to
    // act as Pseudo-Random Generator (PRG) for
    // the KEM (Key Encapsulation Method/Mechanism) generation to
    // be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public SecureRandom get_secure_pseudo_random_generator_kem_generation() {
            
        // Return of the Java's Secure Random to
        // act as Pseudo-Random Generator (PRG) for
        // the KEM (Key Encapsulation Method/Mechanism) generation to
        // be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.secure_pseudo_random_generator_kem_generation;
    
    }
        
    
    // Method to initialize the generation of the instance of
    // the KEM (Key Encapsulation Method/Mechanism) to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public void initialize_key_encapsulation_method_generation() {
       
        // Initialization of the generation of the instance of
        // the KEM (Key Encapsulation Method/Mechanism) to be
        // used by the CRYSTALS-Kyber Public-Key Cryptosystem. 
        this.key_encapsulation_method_generation = 
            new KyberKEMGenerator( this.secure_pseudo_random_generator_kem_generation );
        
    }
        
    
    // Method to initialize the extraction of the instance of
    // the KEM (Key Encapsulation Method/Mechanism) to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public void initialize_key_encapsulation_method_extraction() {
       
        // Initialization of the extraction of the instance of
        // the KEM (Key Encapsulation Method/Mechanism) to be
        // used by the CRYSTALS-Kyber Public-Key Cryptosystem. 
        this.key_encapsulation_method_generation = 
            new KyberKEMGenerator( this.secure_pseudo_random_generator_kem_generation );
        
    }
    
    
    public SecretWithEncapsulation encapsulate_symmetric_secret_key() {
        
        if( this.key_encapsulation_method_generation != null ) {
        
            if( this.get_public_key_shared_payload() != null ) {
            
                return this.key_encapsulation_method_generation
                           .generateEncapsulated( this.get_public_key_shared_payload() ); // TODO change this

            }
            
        }
        else {
            
            return null;
            
        }
        
    }
    
    
    public byte[] get_symmetric_secret_key_bytes_from_encapsulation() {
        
        if( this.key_encapsulation_method_generation != null ) {
        
            return this.encapsulate_symmetric_secret_key().getSecret();
            
        }
        else {
            
            return null;
            
        }
        
    }
    
    
    public int get_symmetric_secret_key_size_bytes_from_encapsulation() {
        
        return this.get_symmetric_secret_key_bytes_from_encapsulation().length;
        
    }
    
    
    public int get_symmetric_secret_key_size_bits_from_encapsulation() {
        
        return ( this.get_symmetric_secret_key_size_bytes_from_encapsulation() * BYTE_SIZE_IN_BITS );
        
    }
    
    
    public byte[] get_encapsulated_symmetric_secret_key_bytes_from_encapsulation() {
        
        if( this.key_encapsulation_method_generation != null ) {
        
            return this.encapsulate_symmetric_secret_key().getEncapsulation();
            
        }
        else {
            
            return null;
            
        }
        
    }
    
    
    public int get_encapsulated_symmetric_secret_key_size_bytes_from_encapsulation() {
        
        return this.get_encapsulated_symmetric_secret_key_bytes_from_encapsulation().length;
        
    }
    
    
    public int get_encapsulated_symmetric_secret_key_size_bits_from_encapsulation() {
        
        return ( this.get_encapsulated_symmetric_secret_key_size_bytes_from_encapsulation() * BYTE_SIZE_IN_BITS );
        
    }
    
    
    
    public byte[] decapsulate_encapsulated_symmetric_secret_key_bytes() {
        
        byte[] encapsulated_symmetric_secret_key_bytes;
        
        if( this.key_encapsulation_method_extraction != null ) {
        
            return this.key_encapsulation_method_extraction
                        .extractSecret( null ); // TODO change this
            
        }
        else {
            
            return null;
            
        }
        
    }
    
    
    // Method to receive and swg of the payload of
    // the public key shared by the other party and
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
    // as a proper object of public key of the cryptosystem. 
    public void receive_and_set_public_key_shared_payload
        ( byte[] public_key_shared_encoding_bytes ) {
        
        // Retrieval of the cipher parameters from
        // the key generation parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem.
        KyberParameters crystals_kyber_cryptosystem_parameters = 
            this.get_key_generation_parameters().getParameters();
        
        // Initialization of the payload of the public key
        // shared by the other party and to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // as a proper object of public key of the cryptosystem. 
        this.public_key_shared_payload = 
            new KyberPublicKeyParameters( crystals_kyber_cryptosystem_parameters,
                                         public_key_shared_encoding_bytes );
        
    }
    
    
    // Method to obtain the payload of the public key
    // shared by the other party and to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem,
    // as a proper object of public key of the cryptosystem. 
    public KyberPublicKeyParameters get_public_key_shared_payload() {
        
        // Return of the payload of the public key
        // shared by the other party and to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // as a proper object of public key of the cryptosystem.
        return this.public_key_shared_payload;
        
    }
    
    
    // Method to obtain the encoding of the symmetric
    // secret key exchanged using the CRYSTALS-Kyber
    // Public-Key Cryptosystem, as a byte array.
    public byte[] symmetric_secret_key_exchanged_encoded_bytes() {
        
        // Return of the encoding of the symmetric
        // secret key exchanged using the CRYSTALS-Kyber
        // Public-Key Cryptosystem, as a byte array.
        return this.symmetric_secret_key_exchanged_encoded_bytes;
        
    }
    
}


// Setting of the parameter choice for the generation of
// the asymmetric key pair to be used internally by 
// the CRYSTALS-Kyber Public-Key Cryptosystem.
final short PARAMETERS_SET_CHOICE_INDEX = 0;


// Creation of a CRYSTALS-Kyber Public-Key Cryptosystem,
// using a set of security parameters pre-defined.
crystals_kyber_cryptosystem crystals_kyber = 
    new crystals_kyber_cryptosystem( PARAMETERS_SET_CHOICE_INDEX );


// Retrieval of the string with the name and
// description of the CRYSTALS-Kyber Public-Key Cryptosystem's
// Security Parameters defined for the key pair generation
// (consisting on private and public keys), as well as
// the respective Key Encapsulation Method/Mechanism (KEM).
String crystals_kyber_security_parameters_name_and_description = 
    crystals_kyber.get_security_parameters_name_and_description();


// Retrieval of the CRYSTALS-Kyber Public-Key
// Cryptosystem's Security Parameters Specifications.
KyberParameterSpec crystals_kyber_security_parameters_specification =
    crystals_kyber.get_security_parameters_specifications();



// Print of the relevant information for the use of
// the CRYSTALS-Kyber Public-Key Cryptosystem.

// Print of two blank lines, as separators.
System.out.println("");
System.out.println("");


// Print of the header of the example of a possible
// execution of the CRYSTALS-Kyber Public-Key Cryptosystem.
System.out.println(" ---------------- EXAMPLE OF AN EXECUTION OF THE CRYSTALS-KYBER " + 
                                      "PUBLIC-KEY CRYPTOSYSTEM ---------------- ");


// Print of two blank lines, as separators.
System.out.println("");
System.out.println("");


// Print of the string with the name and
// description of the CRYSTALS-Kyber Public-Key Cryptosystem's
// Security Parameters defined for the key pair generation
// (consisting on private and public keys), as well as
// the respective Key Encapsulation Method/Mechanism (KEM).
System.out.println( "NAME AND DESCRIPTION OF THE SECURITY PARAMETERS ADOPTED:\n" +
                    crystals_kyber_security_parameters_name_and_description );


// Print of two blank lines, as separators.
System.out.println("");
System.out.println("");


// Retrieval of the set name of the CRYSTALS-Kyber
// Public-Key Cryptosystem's Security Parameters Specifications.
String crystals_kyber_security_parameters_specification_name =
    crystals_kyber_security_parameters_specification.getName();

// Print of the set name of the CRYSTALS-Kyber Public-Key
// Cryptosystem's Security Parameters Specifications.
System.out.println( "NAME OF THE SECURITY PARAMETERS SPECIFICATION:\n=> " +
                    crystals_kyber_security_parameters_specification_name );

org.bouncycastle.pqc.crypto.crystals.kyber.KyberEngine is not public in org.bouncycastle.pqc.crypto.crystals.kyber; cannot be accessed from outside package: org.bouncycastle.pqc.crypto.crystals.kyber.KyberEngine is not public in org.bouncycastle.pqc.crypto.crystals.kyber; cannot be accessed from outside package